In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

df = pd.read_csv("sales_pos.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


# Q1

In [112]:
df_q1 = df[["prod", "job", "purchase"]]
prod = df_q1.groupby(["prod"])["purchase"].sum().sort_values(ascending=False).index[0] # P00025442
df_q1[df_q1["prod"] == prod].groupby(["job"])["purchase"].sum().sort_values(ascending=False).index[0] # 해당 상품의 개수를 ㅜㄱ하는 것이므로 우연히 답은 맞았으나 이건 틀린 것


4

In [110]:
#답
df_q1 = df[["prod", "job", "purchase"]].copy()
best_prod = df_q1.groupby("prod")["purchase"].sum().idxmax()
df_q1_sub = df_q1.loc[df_q1["prod"] == best_prod, ]
df_q1_sub["job"].value_counts().idxmax()

4

# Q2

In [113]:
# df_q2 = df[df["age_group"] == "26-35"].copy()
df_q2 = df.copy()

df_q2["prod_cat1"] = df_q2["prod_cat1"].fillna("0").astype(int).astype(str)
df_q2["prod_cat2"] = df_q2["prod_cat2"].fillna("0").astype(int).astype(str)
df_q2["prod_cat3"] = df_q2["prod_cat3"].fillna("0").astype(int).astype(str)
df_q2["prod_cat"] = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]
# df_q2[df_q2["user"] == 1].groupby(["prod_cat"]).count().reset_index()
temp = df_q2.groupby(["user", "prod_cat"]).count().reset_index()
temp[temp["user"] == 1].shape[0] # 21개 맞게 나옴 



21

In [128]:
#답
df_u1 = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]]
df_u1 = df_u1.reset_index(drop=True)
df_u1 = df_u1.fillna(0) # pandas 는 결측치가 있으면 float64로 변경된다.
df_u1.drop_duplicates().shape # 21 개 나온다.

df_u1["prod_cat1"] = df_u1["prod_cat1"].astype("int").astype("str")
df_u1["prod_cat2"] = df_u1["prod_cat2"].astype("int").astype("str")
df_u1["prod_cat3"] = df_u1["prod_cat3"].astype("int").astype("str")
df_u1["prod_cat"] = df_u1["prod_cat1"] + "-" + df_u1["prod_cat2"] + "-" + df_u1["prod_cat3"]
df_u1["prod_cat"].nunique() # 21개 나온다.

df_q2 = df.loc[df["age_group"] == "26-35", ]
df_q2 = df_q2[["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]].reset_index(drop=True).fillna(0)


df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("int").astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2["prod_cat"] = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]

# groupby Try
df_q2_agg = df_q2.groupby(["user", "marital"])["prod_cat"].nunique().reset_index()
df_q2_agg

stat_m0 = df_q2_agg.loc[df_q2_agg["marital"] == 0, "prod_cat"].mean()
stat_m1 = df_q2_agg.loc[df_q2_agg["marital"] == 1, "prod_cat"].mean()
round(abs(stat_m0 - stat_m1), 2) # 0.13



0.13

# Q3

In [ ]:
df_q3 = df[["gender", "age_group", "job", "city", "marital", "prod", "purchase"]]

In [159]:
# 답

df_q3_user = df[["user", "gender", "age_group", "job", "city", "marital" ]].copy()
df_q3_user["user"].nunique() == df_q3_user.drop_duplicates().reset_index(drop = True).shape[0] # 둘다 5891 로 동일하다, 즉, user 한명당 하나의 데이터만 가지고 있다.

df_q3_agg = df.groupby("user")[["prod", "purchase"]].agg({"prod": "nunique", "purchase": "sum"})
df_q3 = df_q3_user.merge(df_q3_agg, on = "user").drop_duplicates().reset_index(drop = True)
df_q3

# Alternative - 위에서 user 별 서로 다른 정보가 없어서 가능한것.. user나 다른걸로 보나.. 
# cols = ["user", "gender", "age_group", "job", "city", "marital" ]
# df_q3 = df.groupby(cols)[["prod", "purchase"]].agg({"prod": "nunique", "purchase": "sum"}).reset_index()
# df_q3

df_q3["gender"] = df_q3["gender"].replace({"M": 1, "F": 0})
arr_u = df_q3["age_group"].unique()
# arr_u = arr_u.sort() # 안된다.
arr_u.sort() 
arr_u

ser_repl = pd.Series(range(7), index = arr_u)

pd.crosstab(df_q3["age_group"], df["age_group"].replace(ser_repl)) # 잘 매칭이 되었는지 검증해보는것

df_q3["age_group"] = df["age_group"].replace(ser_repl)

#df_q3_dum = pd.get_dummies(df_q3, dtype="int") # 이렇게 하면 object type은 자동변환 하지만, job 처럼 애초에 int 타입은 변환이 안된다. 문제에서 직업과 도시라고 했으니 지정 필요
df_q3_dum = pd.get_dummies(df_q3, columns=["job", "city"], dtype="int")
df_q3_dum = df_q3_dum.drop(columns=["user"])

# test/train을 나누지 않으므로 모델 필요 없다고 한다.
arr_q3_nor = MinMaxScaler().fit_transform(df_q3_dum)
arr_q3_nor[:1,]


model_kmeans = KMeans(n_clusters=7, random_state=123)
model_kmeans.fit(arr_q3_nor)
len(model_kmeans.labels_) # 5891
round(silhouette_score(X=arr_q3_nor, labels = model_kmeans.labels_), 2)





d:\2.Projects\8. Python\DS2\.venv\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


0.18

In [163]:
df_q3["cluster"] = model_kmeans.labels_
df_q3.groupby("cluster")["age_group"].mean() # 누구의 나이대가 큰지 작은지 확인 가능
df_q3["cluster"].value_counts() # 클러스터의 개수 확인

cluster
2    1291
4    1004
6     980
0     868
1     703
5     652
3     393
Name: count, dtype: int64